In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split

In [110]:
dataTrain = pd.read_csv('../setTrainMap.csv')

In [111]:
dataTrain.head()

,PassengerId,Survived,Age,SibSp,Ticket,Parch,Fare,Sex_female,Pclass_3,Pclass_2,...,Embarked_S,Mr,Mrs,Miss,Master,Dr,OtherTitles,With_Cabin,Old,Kid
0,1,0,22.0,1,21171,0,7.2500,False,True,False,...,True,True,False,False,False,False,False,False,False,False
1,2,1,38.0,1,17599,0,71.2833,True,False,False,...,False,False,True,False,False,False,False,True,False,False
2,3,1,26.0,0,3101282,0,7.9250,True,True,False,...,True,False,False,True,False,False,False,False,False,False
3,4,1,35.0,1,113803,0,53.1000,True,False,False,...,True,False,True,False,False,False,False,True,False,False
4,5,0,35.0,0,373450,0,8.0500,False,True,False,...,True,True,False,False,False,False,False,False,False,False


In [112]:
dataTrain = dataTrain.drop("Embarked_Q",axis=1)
dataTrain = dataTrain.drop("Embarked_C",axis=1)
dataTrain = dataTrain.drop("OtherTitles",axis=1)
dataTrain = dataTrain.drop("Dr",axis=1)
dataTrain = dataTrain.drop("Master",axis=1)
dataTrain = dataTrain.drop("Kid",axis=1)
dataTrain = dataTrain.drop("Old",axis=1)
dataTrain = dataTrain.drop("Pclass_2",axis=1)
dataTrain = dataTrain.drop("Pclass_3",axis=1)
dataTrain = dataTrain.drop("Parch",axis=1)
dataTrain = dataTrain.drop("Pclass_1",axis=1)
dataTrain = dataTrain.drop("Embarked_S",axis=1)
dataTrain = dataTrain.drop("Mrs",axis=1)
dataTrain = dataTrain.drop("Miss",axis=1)
dataTrain = dataTrain.drop("SibSp",axis=1)
dataTrain = dataTrain.drop("With_Cabin",axis=1)

In [113]:
dataTrain = dataTrain.drop("PassengerId",axis=1)

In [114]:
dataPositivos = dataTrain[dataTrain['Survived'] == 1 ]
dataNegativos = dataTrain[dataTrain['Survived'] == 0 ].head(342)
dataTrainBalanceado = pd.concat([dataNegativos,dataPositivos])

In [115]:
X_train, X_test, y_train, y_test = train_test_split(dataTrainBalanceado.drop(['Survived'], axis=1).values, dataTrainBalanceado["Survived"] , test_size=0.20, random_state=75)


In [116]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 3000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 7)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True)
# Minimum number of samples required at each leaf node
min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True)
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               "min_samples_split": np.arange(1,150,5),
               "min_samples_leaf": np.arange(1,60,5),
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 228, 256, 284, 313, 341, 369, 397, 426, 454, 482, 511, 539, 567, 595, 624, 652, 680, 709, 737, 765, 793, 822, 850, 878, 907, 935, 963, 991, 1020, 1048, 1076, 1105, 1133, 1161, 1189, 1218, 1246, 1274, 1303, 1331, 1359, 1387, 1416, 1444, 1472, 1501, 1529, 1557, 1585, 1614, 1642, 1670, 1698, 1727, 1755, 1783, 1812, 1840, 1868, 1896, 1925, 1953, 1981, 2010, 2038, 2066, 2094, 2123, 2151, 2179, 2208, 2236, 2264, 2292, 2321, 2349, 2377, 2406, 2434, 2462, 2490, 2519, 2547, 2575, 2604, 2632, 2660, 2688, 2717, 2745, 2773, 2802, 2830, 2858, 2886, 2915, 2943, 2971, 3000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': array([  1,   6,  11,  16,  21,  26,  31,  36,  41,  46,  51,  56,  61,
        66,  71,  76,  81,  86,  91,  96, 101, 106, 111, 116, 121, 126,
       131, 136, 141, 146]), 'min_samples_leaf': array([ 1,  6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]), 'bootstrap': [True, False]}


In [117]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.metrics import make_scorer, roc_auc_score
auc = make_scorer(roc_auc_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [118]:
rf_random.best_params_


{'n_estimators': 369,
 'min_samples_split': 31,
 'min_samples_leaf': 11,
 'max_features': 'sqrt',
 'max_depth': 76,
 'bootstrap': False}

In [119]:
clf = RandomForestClassifier(n_estimators = 369,
min_samples_split= 31,
min_samples_leaf= 11,
max_features = 'sqrt',
max_depth= 76,
bootstrap= False)

In [120]:
clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=76, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=11, min_samples_split=31,
                       min_weight_fraction_leaf=0.0, n_estimators=369,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [121]:
result = clf.predict(X_test)

In [122]:
from sklearn import metrics
pred = result
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.8193485763589301

In [123]:
(pred == y_test).mean()

0.8175182481751825

In [124]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 4 (0.362044)
2. feature 3 (0.246818)
3. feature 1 (0.153776)
4. feature 2 (0.152869)
5. feature 0 (0.084492)


In [125]:
dataTrainBalanceado.columns

Index(['Survived', 'Age', 'Ticket', 'Fare', 'Sex_female', 'Mr'], dtype='object')

In [126]:
dataTest = pd.read_csv('../testMap.csv')


In [83]:
dataTest.head()

,PassengerId,Age,SibSp,Ticket,Parch,Fare,Sex_female,Pclass_3,Pclass_2,Pclass_1,...,Embarked_S,Mr,Mrs,Miss,Master,Dr,OtherTitles,With_Cabin,Old,Kid
0,892,34.5,0,330911,0,7.8292,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
1,893,47.0,1,363272,0,7.0000,True,True,False,False,...,True,False,True,False,False,False,False,False,False,False
2,894,62.0,0,240276,0,9.6875,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
3,895,27.0,0,315154,0,8.6625,False,True,False,False,...,True,True,False,False,False,False,False,False,False,False
4,896,22.0,1,3101298,1,12.2875,True,True,False,False,...,True,False,True,False,False,False,False,False,False,False


In [84]:
dataTest["Fare"] = dataTest["Fare"].fillna(0)

In [85]:
dataTest.columns

Index(['PassengerId', 'Age', 'SibSp', 'Ticket', 'Parch', 'Fare', 'Sex_female',
       'Pclass_3', 'Pclass_2', 'Pclass_1', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Mr', 'Mrs', 'Miss', 'Master', 'Dr', 'OtherTitles',
       'With_Cabin', 'Old', 'Kid'],
      dtype='object')

In [87]:
dataTest = dataTrain.drop("Embarked_Q",axis=1)
dataTest = dataTrain.drop("Embarked_C",axis=1)
dataTest = dataTrain.drop("OtherTitles",axis=1)
dataTest = dataTrain.drop("Dr",axis=1)
dataTest = dataTrain.drop("Master",axis=1)
dataTest = dataTrain.drop("Kid",axis=1)
dataTest = dataTrain.drop("Old",axis=1)
dataTest = dataTrain.drop("Pclass_2",axis=1)
dataTest = dataTest.drop("Pclass_3",axis=1)
dataTest = dataTest.drop("Parch",axis=1)
dataTest = dataTest.drop("Pclass_1",axis=1)
dataTest = dataTest.drop("Embarked_S",axis=1)
dataTest = dataTest.drop("Mrs",axis=1)
dataTest = dataTest.drop("Miss",axis=1)
dataTest = dataTest.drop("SibSp",axis=1)
dataTest = dataTest.drop("With_Cabin",axis=1)

In [88]:
passengerId = dataTest["PassengerId"]
dataTest = dataTest.drop("PassengerId",axis=1)

In [89]:
result = clf.predict(dataTest)

In [90]:
result

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [91]:
dataTest["PassengerId"] = passengerId
dataTest["Survived"] = result

In [93]:
dataTest[["PassengerId","Survived"]].to_csv('submitRFBalanceado.csv',index = False)
